In [41]:
# 国际惯例，先导入相应的包，并且重命名
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt     

import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly import tools

首先看看数据如何

In [42]:
data=pd.read_csv('ex2/ex2data1.txt', names=['exm1', 'exm2','result'])
data.head()

,exm1,exm2,result
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


再看看统计数据情况

In [43]:
data.describe()

,exm1,exm2,result
count,100.000000,100.000000,100.000000
mean,65.644274,66.221998,0.600000
std,19.458222,18.582783,0.492366
min,30.058822,30.603263,0.000000
25%,50.919511,48.179205,0.000000
50%,67.032988,67.682381,1.000000
75%,80.212529,79.360605,1.000000
max,99.827858,98.869436,1.000000


## 1.1 可视化数据

把数据可视化

In [44]:
T = data[data['result'] == 1]
F = data[data['result'] == 0]

fig = px.scatter(x=data.exm1, y=data.exm2, symbol=data.result,color=(data.result*-0.8),height=600,width=1000)
fig.update_layout(xaxis = dict(range = [0,130]), yaxis = dict(range = [0,130]))
fig.show()


实现Sigmoid函数
$$
g(z)=\frac{1}{1+e^{-z}}
$$

In [45]:
def sigmoid(z):
    return 1 / ( 1 + np.exp(-z))
    # return 1 if (1 / ( 1 + np.exp(-z)))>=0.5 else 0   
    #np.exp输入为数组时，可返回数组对应所有值的结果，即结果也为同维度的数组，在这里用于向量计算

可以看到当输入值大于 `36` 之后，计算结果已经为 `1`


In [46]:
print(1 / ( 1 + np.exp(-36)),' ',1 / ( 1 + np.exp(-37)))
print(sigmoid(36),' ',sigmoid(37))

0.9999999999999998   1.0
0.9999999999999998   1.0


看看Sigmoid的曲线

In [47]:
t_X = np.arange(-50, 50, 0.1)
fig = px.line(x=t_X,y=sigmoid(t_X),height=600, width=1000)
fig.show()

## 1.2 梯度下降法

逻辑回归选用新的代价函数
$$
J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{Cost(h_{\theta }(x),y)} \\
$$

$$
Cost(h_{\theta }(x),y) = \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}} \right)
$$

$$
Cost(h_{\theta }(x),y) = 
\begin{cases}
-log(h_{\theta }(x))\quad\quad\quad, y=1 \\
-log(1-h_{\theta }(x))\quad\quad,y=0
\end{cases}
$$

优化代价函数使得其成为一个公式：
$$
Cost(h_{\theta }(x),y) = -y*log(h_{\theta }(x)) -(1-y)*log(1-h_{\theta }(x))
$$

In [48]:
def costFunction(theta, X, y):
    theta = np.matrix(theta)
    part1 = np.multiply(-y,np.log(sigmoid(X @ theta.T)))     # X @ theta 相当于矩阵乘法
    part2 = np.multiply(1-y,np.log(1-sigmoid(X @ theta.T)))

    return np.mean(part1 - part2)

处理初始数据,并检查格式

In [49]:
if 'ones' not in data.columns:
    data.insert(0,'ones',1)

X = np.matrix(data.loc[:,['ones','exm1','exm2']])
y = np.matrix(data.loc[:,['result']].values)

theta = np.zeros(X.shape[1])

X.shape,y.shape,theta.shape

((100, 3), (100, 1), (3,))

#### 迭代参数
$${{\theta }_{j}}:={{\theta }_{j}}-\alpha \frac{\partial }{\partial {{\theta }_{j}}}J\left( \theta  \right)$$
求导：
$${{\theta }_{j}}:={{\theta }_{j}}-\alpha\frac{1}{m}\sum\limits_{i=1}^{m}{{{\left(\left( {{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}} \right)x_j^{(i)}\right)}}}$$

In [50]:
def GD(X, y, theta, alpha, iters):
    theta = np.matrix(theta)
    theta_temp = np.matrix(np.zeros(theta.shape))  # 存储每一轮迭代的theta参数
    history_cost = np.zeros(iters)

    for i in range(iters):
        distance = sigmoid(X @ theta.T) - y
        theta_temp = theta_temp - alpha*np.mean(np.multiply(distance,X),axis=0)

        theta = theta_temp
        history_cost[i] = costFunction(theta, X, y)

    return theta, history_cost
        

初始化参数，计算初始代价

In [51]:
iters=50000
alpha=0.001

costFunction(theta, X, y)

0.6931471805599453

In [52]:
g, history_cost = GD(X, y, theta, alpha, iters)
g

matrix([[-2.84946254,  0.03070169,  0.0227595 ]])

In [53]:

diedai= np.linspace(1, iters, iters)    #制作迭代次数数据
fig = px.scatter(x=diedai,y=history_cost,height=600,width=1000)
fig.show()

## 1.3 高级优化法 学习$\theta$参数
  
  在Octave中使用`fminunc`来优化函数、计算成本和梯度参数，而在Python中用`scipy.optimize`来替代，  
  关于`scipy.optimize`的用法也对cost函数有一定要求，要求theta作为第一参数，且theta必须为一个数组格式  
  即shape为(n,)的格式，需要注意  
  具体使用参考以下文章
  ```
  https://www.jb51.net/article/181639.htm
  ```

首先重写一个<font color='#db692c' style=''>**计算梯度**</font>的方法

In [54]:
def cul_Gradient(theta, X, y):
    theta = np.matrix(theta)
    return np.mean(np.multiply(sigmoid(X @ theta.T) - y,X),axis=0)

In [55]:
cul_Gradient(theta, X, y)

matrix([[ -0.1       , -12.00921659, -11.26284221]])

#### 调用`scipy.optimize`

In [56]:
import scipy.optimize as opt

result = opt.fmin_tnc(func=costFunction, x0=theta, fprime=cul_Gradient, args=(X, y))
result

(array([-25.16131863,   0.20623159,   0.20147149]), 36, 0)

计算优化后的参数代价

In [57]:
theta = result[0]
costFunction(theta, X, y)

0.20349770158947458

### 1.4 评论逻辑回归
  
完成$\theta$参数的学习，即将用得到的参数进行预估学生录取情况  
  
通过用我们的训练数据来计算我们所得模型的精准度  
  
逻辑回归模型的假设函数：
$$h_θ(x) = \frac{1}{1+e^{-θ^TX}}$$
  
当$h_θ$大于等于0.5时，预测 y=1

当$h_θ$小于0.5时，预测 y=0 。

In [58]:
def predict(theta, X):

    predictions = sigmoid(X @ np.matrix(theta).T)
    return np.rint(predictions)

In [59]:
# 计算预测精度
predictions = predict(theta, X)
correct = [1 if a==b else 0 for (a, b) in zip(predictions, y)]
accuracy = sum(correct) / len(X)
print('精确度为：',accuracy)

精确度为： 0.89


### 绘制决策边界
  
该条边界满足：
$$
\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2} = 0
$$

In [60]:
x1 = np.arange(130, step=0.1)
x2 = -(theta[0] + theta[1]*x1) / theta[2]

fig = px.scatter(x=data.exm1, y=data.exm2, symbol=data.result,color=(data.result*-0.8),height=600,width=1000)
fig.add_trace(go.Scatter(x=x1, y=x2, mode='lines'))
fig.update_layout(xaxis = dict(range = [10,120]), yaxis = dict(range = [10,120]))
fig.show()

### 1.5 多元分类

在面对多元分类需求时，可用多个分类器对每一个需要区分类型训练分类，选出分类效果最好的，概率最高的一个
![](https://cdn.jsdelivr.net/gh/YINGHAIDADA/imageStorege@main/image/1651395095773-2022-5-116:51:36.png)

# 2. 正则化逻辑回归